# Cluster and HII-regions Single <a class="tocSkip">

the aim of this notebook is to combine the HII-region and cluster catalogues. This notebook is for use with a single galaxy at a time.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column
from pnlf.plot import quick_plot, add_scale

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logging.basicConfig(stream=sys.stdout,datefmt='%H:%M:%S',level=logging.INFO)
logger = logging.getLogger(__name__)

basedir = Path('..')  # where we save stuff (and )
data_ext = Path('a:') # raw data

# we use the sample table for basic galaxy properties
sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('Name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])

## Read in data

the galaxies listed in `hst_sample` have a cluster catalogue. The galaxies listed in `muse_sample` have astrosat observations to measure the FUV.

In [ ]:
hst_sample      = set(['NGC0628', 'NGC1433', 'NGC1566', 'NGC3351', 'NGC3627', 'NGC4535'])
astrosat_sample = set([x.stem.split('_')[0] for x in (data_ext/'Astrosat').iterdir() if x.is_file() and x.suffix=='.fits'])
muse_sample     = set(sample_table['Name'])
complete_sample = hst_sample & astrosat_sample & muse_sample

name = 'NGC1566'

### MUSE (DAP + nebulae catalogues)

In [ ]:
from pnlf.auxiliary import filter_table
from pnlf.io import ReadLineMaps

p = {x:sample_table.loc[name][x] for x in sample_table.columns}

# DAP linemaps (Halpha and OIII)
filename = data_ext / 'MUSE_DR2' / 'MUSEDAP' / f'{name}_MAPS.fits'
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header)) 

path = data_ext / 'MUSE_DR2' / 'filterImages' 
sdss_g, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_g_WCS_Pall_mad.fits',header=True)
sdss_r, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_r_WCS_Pall_mad.fits',header=True)
sdss_i, h = fits.getdata(path / f'{name}_IMAGE_FOV_SDSS_i_WCS_Pall_mad.fits',header=True)
    
# nebulae catalogue from Francesco (mostly HII-regions)
filename = basedir / 'data' / 'interim' / f'Nebulae_Catalogue_with_FUV_DR2.fits'
with fits.open(filename) as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg)
nebulae.rename_columns(['cen_x','cen_y'],['x','y'])
nebulae.add_index('region_ID')

with np.errstate(divide='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    SIII = nebulae['SIII6312_FLUX_CORR']+nebulae['SIII9068_FLUX_CORR']
    nebulae[SII>0]['[SIII]/[SII]'] = SIII[SII>0]/SII[SII>0]
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX_CORR']/nebulae['FUV_FLUX_CORR']
    
HII_regions = filter_table(nebulae,gal_name=name,BPT_NII=0,BPT_SII=0,BPT_OI=0)
nebulae = filter_table(nebulae,gal_name=name)

filename = data_ext / 'MUSE_DR2' / 'Nebulae catalogue' /'spatial_masks'/f'{name}_HIIreg_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data-1,mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# WFI image (larger FOV)
filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
with fits.open(filename) as hdul:
    WFI = NDData(data=hdul[0].data,
                 meta=hdul[0].header,
                 wcs=WCS(hdul[0].header))
    
# most of the time we do not need the datacubes
if False:
    #from spectral_cube import SpectralCube
    filename = Path('g:') /'Archive'/'MUSE'/'DR2'/'datacubes'/f'{name}_DATACUBE_FINAL_WCS_Pall_mad.fits'
    with fits.open(filename , memmap=True, mode='denywrite') as hdul:
        data_cube   = hdul[1].data
        cube_header = hdul[1].header   
    
print(f'{name}: {len(HII_regions)} HII-regions in final catalogue')

### HST

**white light + filter images**

In [ ]:
# whitelight image (we set 0s to nan)
with fits.open(data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits') as hdul:
    hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    hst_whitelight.data[hst_whitelight.data==0] = np.nan
    
# filter image with uncertainties
filename = data_ext / 'HST' / 'filterImages' / f'{name.lower()}_uvis_f275w_exp_drc_sci.fits'
with fits.open(filename) as hdul:
    F275 = NDData(hdul[0].data,
                  mask=hdul[0].data==0,
                  meta=hdul[0].header,
                  wcs=WCS(hdul[0].header))
filename = data_ext / 'HST' / 'filterImages' / f'{name.lower()}_uvis_f275w_err_drc_sci.fits'
with fits.open(filename) as hdul:
    F275.uncertainty = StdDevUncertainty(hdul[0].data)

**cluster and associations catalogues**

the following 7 galaxies will be available (end of february)

NGC 628, NGC 1365, NGC 1433, NGC 1566, NGC 3351, NGC 3627, NGC 4535

In [ ]:
# select resolution for association catlaogue
res = 32

# cluster catalogues
filename = data_ext / 'HST' / 'cluster catalogue' / f'{name}_phangshst_base_catalog.fits'
with fits.open(filename) as hdul:
    clusters = Table(hdul[1].data)
clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.degree,clusters['PHANGS_DEC']*u.degree)

# remove LEGUS columns
clusters = clusters[[x for x in clusters.columns if 'LEGUS' not in x]]
# remove the PHANGS label from the column names
clusters.rename_columns([x for x in clusters.columns],[x.replace('PHANGS_','') for x in clusters.columns])
# we rename the columns such that the clusters and associations are identical
clusters.rename_columns(['ID_CLUSTERS_V0_9','AGE_MINCHISQ','MASS_MINCHISQ','EBV_MINCHISQ',
                         'AGE_MINCHISQ_ERR','MASS_MINCHISQ_ERR','EBV_MINCHISQ_ERR'],
                        ['cluster_ID','age','mass','EBV','age_err','mass_err','EBV_err'])
# only CLASS 1,2 and 3 are classified as clusters
clusters = filter_table(clusters,CLUSTER_CLASS=[1,2,3])
print(f'{name}: {len(clusters)} clusters in final catalogue')    

# association catalogue
filename = data_ext / 'HST' / 'associations' / name / 'associations' / f'{name.lower()}_phangshst_associations_nuv_ws{res}pc_v0_9.fits'
with fits.open(filename) as hdul:
    associations = Table(hdul[1].data)
associations['SkyCoord'] = SkyCoord(associations['reg_ra']*u.degree,associations['reg_dec']*u.degree)
# we rename the columns such that the clusters and associations are identical
associations.rename_columns(['reg_id','reg_ra','reg_dec','reg_x','reg_y',
                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Mass_MinChiSq','reg_dolflux_Ebv_MinChiSq',
                             'reg_dolflux_Age_MinChiSq_err','reg_dolflux_Mass_MinChiSq_err','reg_dolflux_Ebv_MinChiSq_err'],
                            ['cluster_ID','RA','DEC','X','Y','age','mass','EBV','age_err','mass_err','EBV_err'])

# associations mask
filename = data_ext / 'HST' / 'associations' / name / 'associations' / f'{name.lower()}_phangshst_associations_nuv_ws{res}pc_idmask_v0_9.fits'
with fits.open(filename) as hdul:
    data = hdul[0].data.astype(float)
    data[data==0] = np.nan
    associations_mask = NDData(data,
                               mask=data==0,
                               meta=hdul[0].header,
                               wcs=WCS(hdul[0].header))
print(f'{name}: {len(associations)} associations in catalogue')    

### Astrosat

https://uvit.iiap.res.in/Instrument/Filters

the resolution is 0.4" per pixel. With a PSF resolution of 1.8" this leads to fwhm ~ 4.5 px. This corresponds to a std = 1.91 px

In [ ]:
# whitelight image
astro_file = data_ext / 'Astrosat' / f'{name}_FUV_F148W_flux_reproj.fits'

if not astro_file.is_file():
    astro_file = data_ext / 'Astrosat' / f'{name}_FUV_F154W_flux_reproj.fits'
    if not astro_file.is_file():
        print(f'no astrosat file for {name}')
    
with fits.open(astro_file) as hdul:
    astrosat = NDData(hdul[0].data,meta=hdul[0].header,wcs=WCS(hdul[0].header))

## Equivalent Width

the first step is to extract the spectra of each HII-region.

Are the spectra continuum subtracted?

In [ ]:
from astropy.visualization import quantity_support
quantity_support()

with fits.open(basedir/'data'/'interim'/f'{name}_VorSpectra.fits') as hdul:
    spectra = Table(hdul[1].data)
    spectral_axis = np.exp(Table(hdul[2].data)['LOGLAM'])*u.Angstrom
    
spectra['region_ID'] = np.arange(len(spectra))
spectra.add_index('region_ID')

In [ ]:
H0 = 67 * u.km / u.s / u.Mpc
z = (H0*Distance(distmod=p['(m-M)'])/c.c).decompose()
lam_HA0 = 6562.8*u.Angstrom
lam_HA = (1+z)*lam_HA0

In [ ]:
from astropy.modeling import models, fitting

def fit_emission_line(spectral_axis,flux,lam0,plot=False):
    
    region = (spectral_axis>lam0-10*u.Angstrom) & (spectral_axis<lam0+15*u.Angstrom) 
    amplitude_guess = np.max(flux[region].value)
    
    # create a model with a gaussian + a constant 
    model = models.Gaussian1D(amplitude_guess, lam0.value, 1) + models.Polynomial1D(degree=0)
    model.amplitude_0.min = 0
    model.mean_0.bounds = (lam0.value-3,lam0.value+3)
    fitter = fitting.LevMarLSQFitter()
    fit = fitter(model, spectral_axis[region].value, flux[region].value)
    
    integrated_flux = fit.amplitude_0*np.sqrt(np.pi)*np.exp(-1/(2*fit.stddev_0**2)) * u.erg/u.s/u.cm**2
    continuum = fit.c0_1 * u.erg/u.s/u.cm**2/u.Angstrom
    eq_width = integrated_flux/continuum
    
    if plot:
        f, ax = plt.subplots()  
        ax.scatter(spectral_axis.value, flux.value,color='black',s=2) 
        ax.plot(spectral_axis[region].value,fit(spectral_axis[region].value),color='tab:red')
        ax.plot(spectral_axis.value,0*spectral_axis.value+fit.c0_1,ls='--',color='tab:red')
        ax.axvline(lam0,color='black',ls='--')
        ax.fill_between(spectral_axis,fit(spectral_axis.value),0*spectral_axis.value+fit.c0_1,alpha=0.5)
        
        eq_region = (spectral_axis>lam0+7*u.Angstrom) & (spectral_axis<lam0+7*u.Angstrom+eq_width) 
        ax.fill_between(spectral_axis[eq_region],0*spectral_axis[eq_region].value+fit.c0_1,0*spectral_axis[eq_region].value,alpha=0.5)
        
        ax.set_xlim(lam0-30*u.Angstrom, lam0+30*u.Angstrom) 
        plt.show()
        
    return fit

region_ID = 12
lam_HA = 6595*u.Angstrom
flux = spectra.loc[region_ID]['SPEC']*u.erg/u.s/u.cm**2/u.A
fit = fit_emission_line(spectral_axis,flux,lam_HA,plot=True)

In [ ]:
lam_HA = 6595*u.Angstrom

HA = []
HII_regions['eq_width'] = np.nan
for region_ID in HII_regions['region_ID']:
    
    flux = spectra.loc[region_ID]['SPEC']*u.erg/u.s/u.cm**2/u.A
    fit = fit_emission_line(spectral_axis,flux,lam_HA,plot=False)
    integrated_flux = fit.amplitude_0*np.sqrt(np.pi)*np.exp(-1/(2*fit.stddev_0**2)) * u.erg/u.s/u.cm**2
    continuum = fit.c0_1 * u.erg/u.s/u.cm**2/u.Angstrom
    eq_width = integrated_flux/continuum
    #if eq_width.value<0: eq_width = np.nan
    HII_regions.loc[region_ID]['eq_width'] = eq_width.value
    
    HA.append(integrated_flux)
    #HA_cat = nebulae.loc[region_ID]['HA6562_FLUX']
    #print(f'{integrated_flux/HA_cat:.2f}')
    #print(f'HA = {fit.mean_0.value:.2f}')
HA = np.array([x.value for x in HA])

In [ ]:
plt.scatter(HA,HII_regions['HA6562_FLUX'])
x = np.linspace(0,1.2e7)
plt.plot(x,1/0.4*x,color='black')
plt.ylim([0,3.2e7])

## HST and MUSE

### compare footprints of different observations

here we compare the footprints of the observations and check which objects overlap. The FOV of astrosat is a circle with a diameter of 28'. This is much larger than HST and MUSE and both will always be covered by the astrosat observations

In [ ]:
from cluster.regions import find_sky_region

reg_muse_pix, reg_muse_sky = find_sky_region(nebulae_mask.mask.astype(int),wcs=nebulae_mask.wcs)
reg_hst_pix, reg_hst_sky = find_sky_region(hst_whitelight.mask.astype(int),wcs=hst_whitelight.wcs)

# check which nebulae/clusters are within the HST/MUSE FOV
associations['in_frame'] = reg_muse_sky.contains(associations['SkyCoord'],nebulae_mask.wcs)
clusters['in_frame'] = reg_muse_sky.contains(clusters['SkyCoord'],nebulae_mask.wcs)
nebulae['in_frame']  = reg_hst_sky.contains(nebulae['SkyCoord'],nebulae_mask.wcs)

print(f'{np.sum(associations["in_frame"])} (of {len(associations)}) associations in MUSE FOV')
print(f'{np.sum(clusters["in_frame"])} (of {len(clusters)}) clusters in MUSE FOV')
print(f'{np.sum(nebulae["in_frame"])} (of {len(nebulae)}) nebulae in HST FOV')

In [ ]:
WFI_cutout = Cutout2D(WFI.data,p['SkyCoord'],size=6*u.arcmin,wcs=WFI.wcs)

# project from muse to hst coordinates
reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

# plot image
ax = quick_plot(WFI_cutout,figsize=(single_column,single_column),cmap=plt.cm.gray)
add_scale(ax,u.arcmin,label="1'",color='white',fontsize=10)

reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE')
reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST')

#ax.set(xlim=[3000,11000],ylim=[3000,11000])
plt.savefig(basedir/'reports'/name/'footpring.pdf',dpi=600)
plt.show()

## Association and nebulae

the association catalogue differs from the clusters in that its entries are extended. Because we match two catalogues with extended objects, we must proceed differently.

In a first step we take a look at a single association

In [ ]:
from skimage.measure import find_contours
from regions import PixCoord, PolygonPixelRegion

cluster_ID = 42
pos = associations['SkyCoord'][associations['cluster_ID']==cluster_ID]

contours = find_contours(associations_mask.data==cluster_ID,0.5,)
coords = max(contours,key=len)

# the coordinates from find_counters are switched compared to astropy
reg_pix  = PolygonPixelRegion(vertices = PixCoord(*coords.T[::-1])) 
reg_sky  = reg_pix.to_sky(associations_mask.wcs)

mask_cutout = Cutout2D(associations_mask.data,pos,size=1*u.arcsecond,wcs=associations_mask.wcs)
F275_cutout = Cutout2D(F275.data,pos,size=2*u.arcsecond,wcs=F275.wcs)

reg_pix_cut  = reg_sky.to_pixel(mask_cutout.wcs)

ax = quick_plot(F275_cutout,figsize=(single_column,single_column),cmap=plt.cm.gray)
ax.imshow(mask_cutout.data,alpha=0.5)

reg_pix_cut.plot(ax=ax,ec='tab:red',label='MUSE')

plt.show()

to match the nebulae to the associations we first reproject the mask of the nebulae to the HST image. We then scale the association mask by the number of associations (assume we have 1432 objects, then 615 becomes 0.0615). This way we can add the two masks together and infer from the resulting unique values which clusters overlap with which associations

In [ ]:
from reproject import reproject_interp
# reproject nebulae mask to hst 
nebulae_hst, _  = reproject_interp(nebulae_mask,
                                   output_projection=associations_mask.wcs,
                                   shape_out=associations_mask.data.shape,
                                   order='nearest-neighbor')    

scale = 10**np.ceil(np.log10(max(associations_mask.data[~np.isnan(associations_mask.data)])))
s_arr = associations_mask.data/scale+nebulae_hst

In [ ]:
# ids of associations, nebulae and combination (sum) of both
a_id = np.unique(associations_mask.data[~np.isnan(associations_mask.data)]).astype(int)
n_id = np.unique(nebulae_hst[~np.isnan(nebulae_hst)]).astype(int)
s_id = np.unique(s_arr[~np.isnan(s_arr)])

# this splits the sum into two parts (nebulae and associations)
a_modf,n_modf = np.modf(s_id)
n_modf = n_modf.astype(int)
a_modf = np.round(a_modf*scale).astype(int)

unique_a, count_a = np.unique(a_modf,return_counts=True)
unique_n, count_n = np.unique(n_modf,return_counts=True)

nebulae_dict = {n : list(a_modf[n_modf==n]) for n in n_id}     
associations_dict = {a : list(n_modf[a_modf==a]) for a in a_id}     

# so far we ensured that the nebulae in unique_n have only one association,
# but it is possible that this association goes beyond the nebulae and into
# a second nebulae. Those objects are excluded here
isolated_nebulae = []
isolated_assoc   = []
for n,v in nebulae_dict.items():
    if len(v)==1:
        if len(associations_dict[v[0]])==1:
            isolated_nebulae.append(n)
            isolated_assoc.append(v[0])
            
print(f'n_associations = {len(a_id)}')
print(f'n_nebulae      = {len(n_id)}')
print(f'1to1 match     = {len(isolated_nebulae)}')

hist,bins,_=plt.hist(count_n,bins=np.arange(0.5,10.5,1),width=0.8)
plt.title('all objects')

plt.grid()
plt.show()

In [ ]:
from cluster.plot import single_cutout

region_ID = 7
position = nebulae['SkyCoord'][nebulae['region_ID']==region_ID]

fig,ax=plt.subplots(figsize=(4,4))
single_cutout(ax=ax,position = position,
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             #label= f'{region_ID}/{nebulae_dict[region_ID][0]}',
             size = 4*u.arcsecond)

#plt.savefig(basedir/'reports'/name/'single_region.png',dpi=315)
plt.show()

plot a cutout and see if the match is correct

### Join catalogues

and make some plots that showcase the position/overlap

In [ ]:
outside = []
for a in isolated_assoc:
    if np.any(np.isnan(nebulae_hst[associations_mask.data==a])):
        outside.append(True)
    else:
        outside.append(False)
outside=np.array(outside)
print(f'{np.sum(~outside)} associations fall completly insde one HII-region')

In [ ]:
from astropy.table import join

# assign to each association the ID of the nebulae 
assoc_tmp = associations[np.searchsorted(associations['cluster_ID'], isolated_assoc)].copy()
assoc_tmp['region_ID'] = isolated_nebulae
assoc_tmp['contained'] = ~outside
nebulae_tmp = HII_regions[np.isin(HII_regions['region_ID'],assoc_tmp['region_ID'])]
if 'eq_width' not in nebulae_tmp.columns:
    nebulae_tmp['eq_width'] = np.nan

catalogue = join(assoc_tmp,nebulae_tmp,keys='region_ID')
catalogue.rename_columns(['X','Y','x','y','RA','DEC','cen_ra','cen_dec','reg_area','region_area',
                          'EBV_1','EBV_2','EBV_err','EBV_ERR','SkyCoord_1','SkyCoord_2'],
                         ['x_asc','y_asc','x_neb','y_neb','ra_asc','dec_asc','ra_neb','dec_neb',
                          'area_asc','area_neb','EBV_stars','EBV_balmer','EBV_stars_err','EBV_balmer_err',
                          'SkyCoord_asc','SkyCoord_neb'])

# select the columns of the joined catalogue
columns = ['cluster_ID','region_ID','x_asc','y_asc','x_neb','y_neb',
           'ra_asc','dec_asc','ra_neb','dec_neb','SkyCoord_asc','SkyCoord_neb','area_asc','area_neb',
           'age','age_err','mass','mass_err','EBV_stars','EBV_stars_err','EBV_balmer','EBV_balmer_err',
           'met_scal','met_scal_err','logq_D91','logq_D91_err',] + \
            [x for x in HII_regions.columns if x.endswith('_FLUX_CORR')] + \
            [x for x in HII_regions.columns if x.endswith('_FLUX_CORR_ERR')] + \
            ['HA/FUV','contained','eq_width']
catalogue = catalogue[columns]
        
catalogue.rename_columns([col for col in catalogue.columns if col.endswith('FLUX_CORR')],
                      [col.replace('FLUX_CORR','flux') for col in catalogue.columns if col.endswith('FLUX_CORR')])
catalogue.rename_columns([col for col in catalogue.columns if col.endswith('FLUX_CORR_ERR')],
                      [col.replace('FLUX_CORR_ERR','flux_err') for col in catalogue.columns if col.endswith('FLUX_CORR_ERR')])
catalogue['cluster_ID'] = catalogue['cluster_ID'].astype('int')
catalogue['region_ID'] = catalogue['region_ID'].astype('int')

catalogue.info.description = 'Joined catalogue between associations and nebulae'
mean_sep = np.mean(catalogue['SkyCoord_asc'].separation(catalogue['SkyCoord_neb']))
print(f'{len(catalogue)} objects in catalogue')
print(f'the mean separation between cluster and association center is {mean_sep.to(u.arcsecond):.2f}')

look at the joined catalogue (containing only nebulae and clusters with a 1 to 1 relation)

In [ ]:
from cluster.plot import multi_cutout

filename = basedir/'reports'/name/f'{name}_isolated_associations_F275'
positions = catalogue['SkyCoord_neb'][catalogue['contained']]
labels = [f'{ri}/{ci}' for ri, ci in catalogue[['region_ID','cluster_ID']][catalogue['contained']]]

multi_cutout(positions = positions,
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             labels= labels,
             size = 4*u.arcsecond,
             filename=filename,
             ncols=5)

create a multi page pdf with all isolated objects

In [ ]:
from cluster.plot import multi_page_cutout

filename = basedir/'reports'/name/f'{name}_isolated_associations_F275'
positions = catalogue['SkyCoord_neb'][catalogue['contained']]
labels = [f'{ri}/{ci}' for ri, ci in catalogue[['region_ID','cluster_ID']][catalogue['contained']]]

multi_page_cutout(positions = positions,
             image = F275,
             mask1 = nebulae_mask,
             mask2 = associations_mask,
             #points= clusters,
             labels= labels,
             size = 4*u.arcsecond,
             filename=filename,
             ncols=5,nrows=4)

In [ ]:
WFI_cutout = Cutout2D(WFI.data,p['SkyCoord'],size=4*u.arcmin,wcs=WFI.wcs)

# project from muse to hst coordinates
reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

# plot image
fig,ax=plt.subplots(figsize=(two_column,two_column),subplot_kw={'projection': WFI_cutout.wcs})
norm = simple_norm(WFI_cutout.data,clip=False,percent=96)
ax.imshow(WFI_cutout.data,norm=norm,origin='lower',cmap=plt.cm.Greys)
add_scale(ax,u.arcmin,label="1'",color='white',fontsize=10)

reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE')
reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST')

x,y = catalogue['SkyCoord_neb'][catalogue['contained']].to_pixel(WFI_cutout.wcs)
ax.scatter(x,y,marker='s',facecolors='none',s=30,lw=1,color='tab:red')

ax.coords[0].set_ticks_visible(False)
ax.coords[1].set_ticks_visible(False)
ax.coords[0].set_ticklabel_visible(False)
ax.coords[1].set_ticklabel_visible(False)
#ax.set(xlim=[3000,11000],ylim=[3000,11000])
plt.show()

export parts of the joined catalogue (right now only the fully contained objects)

In [ ]:
export = catalogue[catalogue['contained']]
export.add_column(export['SkyCoord_asc'].to_string(style='hmsdms',precision=2),index=6,name='RaDec_asc')
export.add_column(export['SkyCoord_neb'].to_string(style='hmsdms',precision=2),index=8,name='RaDec_neb')

for col in export.columns:
    if col not in ['RaDec_asc','RaDec_neb','region_ID','cluster_ID']:
        export[col].info.format = '%.2f'

del export[['ra_asc','dec_asc','ra_neb','dec_neb','SkyCoord_neb','SkyCoord_asc','contained','HA/FUV']]

filename = basedir/'data'/'interim'/f'{name}_associations_and_nebulae_joined.txt'
with open(filename,'w',newline='\n') as f:
    ascii.write(export,f,format='fixed_width_two_line',overwrite=True,delimiter_pad=' ',position_char='=')

### Correlations in joined catalogue

first we create a mask to select a subset of objects (e.g. based on mass)

In [ ]:
from astropy.coordinates import match_coordinates_sky
from cluster.auxiliary import bin_stat

# separation to other associations
idx,sep_others,_= match_coordinates_sky(catalogue['SkyCoord_asc'],associations['SkyCoord'],nthneighbor=2)
idx,sep_int,_= match_coordinates_sky(catalogue['SkyCoord_asc'],catalogue['SkyCoord_neb'])

# size of the association compared to the HII-region
small_HII = (catalogue['area_neb']/0.039) / (catalogue['area_asc']/11.95) > 2

# distance to centre of galaxy
galactic_center = SkyCoord(ra=p['R.A.'],dec=p['Dec.'])
catalogue['galactic_radius'] = catalogue['SkyCoord_neb'].separation(galactic_center).to(u.arcmin)

# define the criteria which objects we use in the plot
criteria = (catalogue['mass']>1e4)  #& (catalogue['age']>catalogue['age_err'])

#& catalogue['contained'] #& (catalogue['galactic_radius']>1*u.arcmin)

print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] #catalogue[criteria]

HA/FUV vs cluster age

In [ ]:
xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(tmp['age'],tmp['HA/FUV']) #,c=tmp['mass'],vmin=1e5,vmax=3e5)
#fig.colorbar(sc,label='mass / Msun')

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/name/f'{name}_HaFUV_over_age.pdf',dpi=600)
plt.show()

equivalent width vs cluster age

In [ ]:
xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(tmp['age'],tmp['eq_width']) #,c=tmp['mass'],vmin=1e5,vmax=3e5)
#fig.colorbar(sc,label='mass / Msun')

x,mean,std = bin_stat(tmp['age'],tmp['eq_width'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='equivalent width / Angstrom',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/name/f'{name}_eq_width_over_age.pdf',dpi=600)
plt.show()

In [ ]:
xlim = [0,100]
criteria = HII_regions['HA/FUV']<150

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(HII_regions['eq_width'][criteria],HII_regions['HA/FUV'][criteria])

x,mean,std = bin_stat(HII_regions['eq_width'][criteria],HII_regions['HA/FUV'][criteria],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='equivalent width / Angstrom',ylabel='Halpha / FUV',xlim=xlim,ylim=(0,70))
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')
plt.savefig(basedir/'reports'/name/f'{name}_HaFUV_over_eq_width.pdf',dpi=600)
plt.show()

Extinction from stars and from nebulae

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# we expect EBV_balmer = 2 EBV_stars

fig = plt.figure(figsize=(single_column,single_column/1.1))
ax = fig.add_subplot()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="10%", pad=0.2,)

xlim = [0,10]
sc=ax.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=plt.cm.viridis_r)
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

plt.savefig(basedir/'reports'/name/f'{name}_EBV_Balmer_vs_Stars.pdf',dpi=600)
plt.show()

### Corner Plot

In [ ]:
from cluster.plot import corner

filename = basedir/'reports'/name/f'{name}_corner'
columns  = ['age','eq_width','HA/FUV','met_scal','logq_D91']
limits   = {'age':(0,10),'eq_width':(0,100),'HA/FUV':(0,50),'met_scal':(8.4,8.7),'logq_D91':(6,8)}

corner(catalogue,columns,limits,filename=filename)

In [ ]:
from scipy.special import comb
from scipy.stats import spearmanr
import itertools

lines = [col for col in catalogue.columns if col.endswith('_flux')]
print(f'{len(lines)} different lines with {comb(len(lines),2)} possible combinations')

correlation = []
for pair in itertools.combinations(lines,2):
    not_nan = ~np.isnan(catalogue[pair[0]]) & ~np.isnan(catalogue[pair[1]])
    r,p = spearmanr(catalogue['age'],catalogue[pair[0]][not_nan]/catalogue[pair[1]][not_nan])
    correlation.append((r,pair))
a = [x for x in correlation if np.abs(x[0])>0.15]
a.sort(key=lambda x: np.abs(x[0]),reverse=True)
a

In [ ]:
catalogue['HA/SII'] = catalogue['HA6562_flux'] / catalogue['SII6716_flux']
catalogue['HA/OI'] = catalogue['HA6562_flux'] / catalogue['OI6300_flux']

catalogue['HA/SII'][~np.isfinite(catalogue['HA/SII'])] = np.nan
catalogue['HA/OI'][~np.isfinite(catalogue['HA/OI'])] = np.nan

Halpha luminosity vs mass

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [1,5e5]
print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] 

fig,ax=plt.subplots(figsize=(6,4))
ax.scatter(tmp['mass'],tmp['HA6562_FLUX'])
x,mean,std = bin_stat(tmp['mass'],tmp['HA6562_FLUX'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='mass / Msun',ylabel='Halpha',xlim=xlim,ylim=[0,500000])

#plt.savefig(basedir/'reports'/name/'HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [1,2e5]
criteria = (catalogue['age']<20) #& (sep>Angle('3"'))
print(f'{np.sum(criteria)} objects match the criteria')
tmp = catalogue[criteria] #catalogue[criteria]

fig,ax=plt.subplots(figsize=(8,6))
ax.scatter(tmp['mass'],tmp['region_area'])
x,mean,std = bin_stat(tmp['mass'],tmp['region_area'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='mass / Msun',ylabel='HII-region area',xlim=xlim)

#plt.savefig(basedir/'reports'/name/'HaFUV_over_age.pdf',dpi=600)
plt.show()

## Clusters and Nebulae

we match the position of the cluster catalogue to the nebulae catalogue

we look up the value (=region_ID) in the spatial nebulae mask for each position in the cluster catalogue and asign this value to the cluster.

In [ ]:
def get_value(data,x,y):
    shape = data.shape
    
    out = []
    for j,i in zip(x,y):
        if 0<i<shape[0] and 0<j<shape[1]:
            out.append(data[int(i),int(j)])
        else:
            out.append(np.nan)
    return out

clusters['in_frame'] = reg_muse_sky.contains(clusters['SkyCoord'],nebulae_mask.wcs)
clusters['MUSE_X'],clusters['MUSE_Y'] = clusters['SkyCoord'].to_pixel(nebulae_mask.wcs)

clusters['region_ID'] = np.nan
clusters['region_ID'][clusters['in_frame']] = get_value(nebulae_mask.data,x=clusters['MUSE_X'][clusters['in_frame']],y=clusters['MUSE_Y'][clusters['in_frame']])
# we exclude clusters in PNe and SNRs 
clusters['region_ID'][~np.isin(clusters['region_ID'],HII_regions['region_ID'])] = np.nan

region_ID = clusters['region_ID']
print(f'{np.sum(~np.isnan(region_ID))} inside and {np.sum(np.isnan(region_ID))} outside of nebulae')
unique, counts = np.unique(region_ID[~np.isnan(region_ID)],return_counts=True)
isolated_clusters = clusters[np.isin(clusters['region_ID'],unique[counts==1])]
print(f'{len(isolated_clusters)} isolated clusters (no other objects in nebulae)')

hist,bins,_=plt.hist(counts,bins=np.arange(0.5,10.5,1),width=0.8)
plt.title('all objects')

plt.grid()
plt.show()

### Plot cutout of single region

this function plots a cutout for one region in different filters. Overplotted are the detected clusters and nebulae (green circles are unclassified clusters and blue circles are classified clusters. Solid regions are HII-regions and dashed regions are other emission line nebulae like PNe or SNRs). 

In [ ]:
# 37,77
position = HII_regions['SkyCoord'][HII_regions['region_ID']==183][0]
#position = isolated_clusters['SkyCoord'][0]
#position = HII_regions['SkyCoord'][np.isin(HII_regions['region_ID'],hst_all_objects['region_ID'])][39]
filename = basedir/'reports'/name/f'{name}_region'

plot_cluster_nebulae(name,position,8*u.arcsec,
                     F275,Halpha,astrosat,
                     sdss_g,sdss_r,sdss_i,
                     reg_hst_sky,nebulae_mask,
                     associations_mask,
                     HII_regions,
                     filename=filename)

For our analysis we require nebulae that are clearly associated with one cluster. Here we plot a cutout for each of those clusters with the detected nebulae and clusters overlayed

In [ ]:
from cluster.plot import plot_cutouts

#nebulae.sort(['HA6562_FLUX'])

sample = isolated_clusters[:20]
sample.sort('region_ID')
filename = basedir/'reports'/name/f'isolated_clusters_F275_{name}'
positions = sample['SkyCoord']
labels = sample['region_ID']

plot_cutouts(positions = positions,
             image = F275,
             mask1 = nebulae_mask,
             points= clusters,
             labels= labels,
             filename=filename,
             ncols=5)


finally we plot the position of those "isolated clusters"

In [ ]:
from pnlf.plot import create_RGB

fig = plt.figure(figsize=(two_column,two_column))
ax1  = fig.add_subplot(111,projection=Halpha.wcs)

gri = create_RGB(sdss_i,sdss_r,sdss_g,percentile=[98,98,98])

# show image of the entire galaxy
ax1.imshow(gri)
reg_hst_muse  = reg_hst_sky.to_pixel(Halpha.wcs)
reg_hst_muse.plot(ax=ax1,ec='tab:red',label='HST',lw=0.5)
x,y = isolated_clusters['SkyCoord'].to_pixel(Halpha.wcs)
ax1.scatter(x,y,marker='s',facecolors='none',s=50,lw=1,color='tab:red')
#plt.savefig(basedir/'reports'/name/f'{name}_location_isolated_clusters.png')
plt.show()

### Join the two catalgoues

run only when something has changed and you want to save a new catalogue

In [ ]:
from astropy.table import join
from cluster.regions import find_sky_region

reg_muse_pix, reg_muse_sky = find_sky_region(nebulae_mask.mask.astype(int),wcs=nebulae_mask.wcs)
    
clusters['in_frame'] = reg_muse_sky.contains(clusters['SkyCoord'],nebulae_mask.wcs)
clusters['MUSE_X'],clusters['MUSE_Y'] = clusters['SkyCoord'].to_pixel(nebulae_mask.wcs)

clusters['region_ID'] = np.nan
clusters['region_ID'][clusters['in_frame']] = get_value(nebulae_mask.data,x=clusters['MUSE_X'][clusters['in_frame']],y=clusters['MUSE_Y'][clusters['in_frame']])
# we exclude clusters in PNe and SNRs 
clusters['region_ID'][~np.isin(clusters['region_ID'],HII_regions['region_ID'])] = np.nan

region_ID = clusters['region_ID']
print(f'{np.sum(~np.isnan(region_ID))} inside and {np.sum(np.isnan(region_ID))} outside of nebulae')
unique, counts = np.unique(region_ID[~np.isnan(region_ID)],return_counts=True)
isolated_clusters = clusters[np.isin(clusters['region_ID'],unique[counts==1])]
print(f'{len(isolated_clusters)} isolated clusters (no other objects in nebulae)')

isolated_nebulae = nebulae[np.isin(nebulae['region_ID'],isolated_clusters['region_ID'])]

catalogue = join(isolated_clusters,isolated_nebulae,keys='region_ID')
#catalogue['met_scal'][0]

del catalogue[['SkyCoord_1','SkyCoord_2']]

# write to file
primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(catalogue)
hdul = fits.HDUList([primary_hdu, table_hdu])
#hdul.writeto(basedir/'data'/'interim'/f'{name}_Clusters+Nebulae.fits',overwrite=True)

### Correlations in combined catalogues

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [0,20]
tmp = catalogue[catalogue['mass']>5e3]

fig,ax=plt.subplots(figsize=(8,6))
ax.scatter(tmp['age'],tmp['HA/FUV'])
x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,fmt='-',color='black')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=[0,20])

#plt.savefig(basedir/'reports'/name/'HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:
filename = basedir/'reports'/f'{name}_age_over_SII.pdf'
fig, ax = plt.subplots(figsize=(single_column,single_column))
ax.scatter(catalogue['age'],catalogue['[%%SVG]'])
ax.set(xlim=(0,12),xlabel='age/Myr',ylabel='Ha/FUV')
plt.show()

## Measure FUV and Halpha at association position

### From point

If I select an aperture smaller than a pixel, the measured flux is directly proportional to the apertuer size. Therefore it doesn't matter that the astrosat resolution is much worse than HST or MUSE

In [ ]:
from dust_extinction.parameter_averages import O94, CCM89

extinction_model = CCM89(Rv=3.1)

def extinction(EBV,EBV_err,wavelength,plot=False):
    '''Calculate the extinction for a given EBV and wavelength with errors'''
    
    EBV = np.atleast_1d(EBV)
    sample_size = 100000

    ext = extinction_model.extinguish(wavelength,Ebv=EBV)
    
    EBV_rand = np.random.normal(loc=EBV,scale=EBV_err,size=(sample_size,len(EBV)))
    ext_arr  = extinction_model.extinguish(wavelength,Ebv=EBV_rand)
        
    ext_err  = np.std(ext_arr,axis=0)
    ext_mean = np.mean(ext_arr,axis=0)
    
    if plot:
        fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))
        ax1.hist(EBV_rand[:,0],bins=100)
        ax1.axvline(EBV[0],color='black')
        ax1.set(xlabel='E(B-V)')
        ax2.hist(ext_arr[:,0],bins=100)
        ax2.axvline(ext[0],color='black')
        ax2.set(xlabel='extinction')
        plt.show()
 
    return ext,ext_err

In [ ]:
from photutils import SkyCircularAperture,SkyCircularAnnulus,aperture_photometry

criteria = np.isin(associations['cluster_ID'],isolated_assoc)

aperture_size = 1*u.arcsecond
positions = associations['SkyCoord'][criteria]

aperture = SkyCircularAperture(positions,aperture_size)

fluxes = associations[['cluster_ID','SkyCoord','age','age_err','mass','mass_err','EBV','EBV_err']][criteria]
fluxes['FUV'] = 1e20*aperture_photometry(astrosat,aperture)['aperture_sum']
fluxes['HA'] = aperture_photometry(Halpha,aperture)['aperture_sum']



# because the HII-regions are sometimes extended and not circular, this is probably not sufficient
'''
r_in,r_out = 5*u.arcsec,8*u.arcsec
A_circle  = np.pi*aperture_size**2
A_annulus = np.pi*(r_out**2-r_in**2)
annulus_aperture = SkyCircularAnnulus(positions,r_in=r_in, r_out=r_out)


fluxes['FUV_bkg'] = 1e20*aperture_photometry(astrosat,annulus_aperture)['aperture_sum']/A_annulus*A_circle
fluxes['HA_bkg'] = aperture_photometry(Halpha,annulus_aperture)['aperture_sum']/A_annulus*A_circle
fluxes['FUV'] = fluxes['FUV']-fluxes['FUV_bkg']
fluxes['HA']  = fluxes['HA']-fluxes['HA_bkg']
'''

# E(B-V) is estimated from nebulae. E(B-V)_star = 0.5 E(B-V)_nebulae. FUV comes directly from stars
extinction_mw  = extinction_model.extinguish(1481*u.angstrom,Ebv=0.5*p['E(B-V)'])
ext_int,ext_int_err = extinction(associations['EBV'][criteria],associations['EBV_err'][criteria],wavelength=1481*u.angstrom)
fluxes['FUV'] = fluxes['FUV'] / extinction_mw 
fluxes['FUV_CORR'] = fluxes['FUV'] / ext_int 

# the Halpha line maps are already MW extinction corrected
ext_int,ext_int_err = extinction(2*associations['EBV'][criteria],associations['EBV_err'][criteria],wavelength=6562*u.angstrom)
fluxes['HA_CORR'] = fluxes['HA'] / ext_int 

In [ ]:
from cluster.auxiliary import bin_stat

bins = 10
xlim=[0.5,10.5]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(fluxes['age'],fluxes['HA']/fluxes['FUV'])

x,mean,std = bin_stat(fluxes['age'],fluxes['HA']/fluxes['FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim,ylim=[-10,125])

plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

bins = 10
xlim=[0.5,10.5]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(fluxes['age'],fluxes['HA_CORR']/fluxes['FUV_CORR'])

x,mean,std = bin_stat(fluxes['age'],fluxes['HA_CORR']/fluxes['FUV_CORR'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim,ylim=[-10,70])


plt.show()

### From mask

because the resolution of MUSE and astrosat is so much worse than HST, many associations won't be resolved and hence we can not measure the fluxes

In [ ]:
from reproject import reproject_interp

associations_muse, _  = reproject_interp(associations_mask,output_projection=Halpha.wcs,shape_out=Halpha.data.shape,order='nearest-neighbor') 
associations_astro, _ = reproject_interp(associations_mask,output_projection=astrosat.wcs,shape_out=astrosat.data.shape,order='nearest-neighbor')    

In [ ]:
sample = list(set(np.unique(associations_muse[~np.isnan(associations_muse)])) & set(np.unique(associations_astro[~np.isnan(associations_astro)])))
sample.sort()
HA_flux = [np.sum(Halpha.data[associations_muse==cluster_ID]) for cluster_ID in sample]
FUV_flux = [np.sum(astrosat.data[associations_astro==cluster_ID]) for cluster_ID in sample]

In [ ]:
from astropy.table import join

fluxes = Table([sample,HA_flux,FUV_flux],names=['cluster_ID','HA','FUV'])
catalogue = join(associations,fluxes[(~np.isnan(HA_flux)) & (~np.isnan(FUV_flux))],keys='cluster_ID')

In [ ]:
bins = 10
xlim=[0,100]
fig,ax=plt.subplots(figsize=(6,6))

ax.scatter(catalogue['age'],catalogue['HA']/catalogue['FUV'])

ax.set(xlim=xlim,xlabel='age/Myr',ylabel='Ha/FUV')
plt.show()

### problem with the resolution

In [ ]:
from reproject import reproject_interp, reproject_exact
from astropy.nddata import block_replicate

nebulae_astrosat, _ = reproject_interp(nebulae_mask,output_projection=astrosat.wcs,shape_out=astrosat.data.shape,order='nearest-neighbor') 
astro_MUSE, _ = reproject_exact(astrosat,output_projection=Halpha.wcs,shape_out=Halpha.data.shape)    
asttro_fine = block_replicate(astrosat,4)

In [ ]:
region_ID = 13
position = nebulae['SkyCoord'][nebulae['region_ID']==region_ID]
size = 4*u.arcsecond

nebulae_cutout       = Cutout2D(nebulae_mask.data,position,size=size,wcs=nebulae_mask.wcs)
astrosat_cutout_MUSE = Cutout2D(astro_MUSE,position,size=size,wcs=Halpha.wcs)

nebulae_cutout_astrosat = Cutout2D(nebulae_astrosat,position,size=size,wcs=astrosat.wcs)
astrosat_cutout = Cutout2D(astrosat.data,position,size=size,wcs=astrosat.wcs)

astrosat_up = block_replicate(astrosat_cutout.data,4)
nebulae_fine, _ = reproject_interp(nebulae_mask,output_projection=astrosat_cutout.wcs,shape_out=astrosat_up.shape,order='nearest-neighbor') 

In [ ]:
from skimage.measure import find_contours

fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,5))

neb_contours = []
for i in np.unique(nebulae_cutout.data[~np.isnan(nebulae_cutout.data)]):
    blank_mask = np.zeros_like(nebulae_cutout.data)
    blank_mask[nebulae_cutout.data==i] = 1
    neb_contours += find_contours(blank_mask, 0.5)
    
neb_contours_astrosat = []
for i in np.unique(nebulae_cutout_astrosat.data[~np.isnan(nebulae_cutout_astrosat.data)]):
    blank_mask = np.zeros_like(nebulae_cutout_astrosat.data)
    blank_mask[nebulae_cutout_astrosat.data==i] = 1
    neb_contours_astrosat += find_contours(blank_mask, 0.5)

neb_contours_fine = []
for i in np.unique(nebulae_fine[~np.isnan(nebulae_fine)]):
    blank_mask = np.zeros_like(nebulae_fine)
    blank_mask[nebulae_fine==i] = 1
    neb_contours_fine += find_contours(blank_mask, 0.5)
    
norm1 = simple_norm(astrosat_cutout.data,clip=False,percent=99)
ax1.imshow(astrosat_cutout.data,norm=norm1)

norm2 = simple_norm(astrosat_cutout_MUSE.data,clip=False,percent=99)
ax2.imshow(astrosat_cutout_MUSE.data,norm=norm2)

norm3 = simple_norm(astrosat_up,clip=False,percent=99)
ax3.imshow(astrosat_up,norm=norm3)

for coords in neb_contours:
    ax2.plot(coords[:,1],coords[:,0],color='tab:blue',lw=0.8)
for coords in neb_contours_astrosat:
    ax1.plot(coords[:,1],coords[:,0],color='tab:red',lw=0.8)   
for coords in neb_contours_fine:
    ax3.plot(coords[:,1],coords[:,0],color='tab:red',lw=0.8)  
    
ax1.set_title('original astrosat resolution')
ax2.set_title('interpolated to MUSE resolution')
    
plt.show()

In [ ]:
print(f'interpolate regions: {np.sum(astrosat_cutout.data[nebulae_cutout_astrosat.data==region_ID]):.2g}')
print(f'interpolate astrosat: {np.sum(np.sum(astrosat_cutout_MUSE.data[nebulae_cutout.data==region_ID])):.2g}')


In [ ]:
from reproject import reproject_exact

ast_small , _  = reproject_exact(astrosat,output_projection=neb_cutout.wcs,shape_out=(101,101)) 
ast_large , _  = reproject_exact(astrosat,output_projection=neb_cutout.wcs,shape_out=(202,202)) 
ast_org = Cutout2D(astrosat.data,position,size=size,wcs=astrosat.wcs)

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(10,5))

ax1.imshow(ast_org.data)
ax2.imshow(ast_small)
ax3.imshow(ast_large)

plt.show()

In [ ]:

arr = np.array([[1,2],[3,4]])


astrosat_upsampled = block_replicate(astrosat,4,conserve_sum=True)


## Compare with SIGNALS

In [ ]:
if name!='NGC0628':
    raise ValueError(f'current Galaxy is {name} and not NGC0628')

from cluster.regions import find_sky_region

reg_muse_pix, reg_muse_sky = find_sky_region(nebulae_mask.mask.astype(int),wcs=nebulae_mask.wcs)

with open(basedir/'data'/'external'/'columns.txt') as f:
    txt = f.read()
names = txt.split('\n')

with fits.open(basedir/'data'/'external'/'NGC628_catalog.fits') as hdul:
    signals = Table(hdul[0].data,names=names)
signals['SkyCoord'] = SkyCoord(signals['RA']*u.degree,signals['DEC']*u.degree)

signals['in_frame'] = reg_muse_sky.contains(signals['SkyCoord'],nebulae_mask.wcs)

In [ ]:
def calculate_ratios(tbl):
    '''
    'log(NII6583/HA)',
    'log(SII6716+6731/HA)',
    'log(SII6716+6731/NII6583)',
    'log(OIII5007/HB)',
    'log(OII3727/HB)',
    'log(OII3727+OII5007/HB)',
    'log(OIII5007/OII3727 )',
    'log(OIII5007/NII6583 )',
    'log(OII3727/NII6583)',
    'log(SII6717/SII6731)'
    ''' 
    
    with np.errstate(divide='ignore'):
        tbl['NII6583/HA'] = np.log10(tbl['NII6583_FLUX']/tbl['HA6562_FLUX'])
        tbl['SII6716+6731/HA'] = np.log10((tbl['SII6716_FLUX']+tbl['SII6730_FLUX'])/tbl['HA6562_FLUX'])
        tbl['SII6716+6731/NII6583'] = np.log10((tbl['SII6716_FLUX']+tbl['SII6730_FLUX'])/tbl['NII6583_FLUX'])
        tbl['OIII5007/HB'] = np.log10(tbl['OIII5006_FLUX']/tbl['HB4861_FLUX'])
        tbl['OIII5007/NII6583'] = np.log10(tbl['OIII5006_FLUX']/tbl['NII6583_FLUX'])
        tbl['SII6717/SII6731'] = np.log10(tbl['SII6716_FLUX']/tbl['SII6730_FLUX'])    

    return tbl

HII_regions = calculate_ratios(HII_regions)

In [ ]:
from astropy.coordinates import match_coordinates_sky, Angle

matchcoord = HII_regions[HII_regions['flag_point_source']==1]
catalogcoord = signals[signals['in_frame']]

idx,sep,_=match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])
crit = sep.__lt__(Angle('0.8"'))

lines = ['log(NII6583/HA)','log(SII6716+6731/HA)','log(SII6716+6731/NII6583)',
             'log(OIII5007/HB)','log(OIII5007/NII6583)','log(SII6717/SII6731)']

for line in lines:
    matchcoord[line] = catalogcoord[idx][line]
matchcoord = matchcoord[crit]

#print(f'SIGNALS in MUSE frame: {np.sum(catalogcoord["in_frame"])}')
print(f'matches {np.sum(crit)}')

In [ ]:
from region import Regions
        
muse_regions = Regions(mask=nebulae_mask.data,projection=nebulae_mask.meta,bkg=-1)

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(projection=Halpha.wcs)

norm = simple_norm(Halpha.data,clip=False,stretch='asinh',percent=90)
ax.imshow(Halpha.data,norm=norm,cmap=plt.cm.Greys)

x,y = catalogcoord[idx]['SkyCoord'].to_pixel(Halpha.wcs)
plt.scatter(x,y,marker='o',fc='none',ec='tab:red',s=2,lw=0.1)

for coords in muse_regions.contours: 
    ax.plot(coords[:,1],coords[:,0],color='tab:blue',lw=0.2)

plt.savefig(basedir/'reports'/name/'signals_nebulae.pdf',dpi=600)

plt.show()

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(10,6))

axes_iter=iter(axes.flatten())

for line in lines:
    ax = next(axes_iter)
    
    ax.scatter(matchcoord[line],matchcoord[line[4:-1]])
    #ax.plot([-3,3],[-3,3])
    ax.set(xlabel=f'{line} SIGNALS',ylabel=f'{line} PHANGS')
plt.tight_layout()
plt.show()

## Starburst99

compare our observations with simulated data

**Note**: the GENEVAHIGH 23 (Z=0.008) model used a metallicity of 0.02 for the high resolution models 

In [ ]:
from starburst.core import Cluster

cluster = Cluster(stellar_model='GENEVAHIGH',metallicity=0.014)

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))

ax1.plot(cluster.ewidth['Equ_width_H_A'],cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],color='black')
sc = ax1.scatter(cluster.ewidth['Equ_width_H_A'],cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],
                 c=cluster.ewidth['Time']/1e6,vmin=0,vmax=10)
fig.colorbar(sc,ax=ax1,label='age / Myr')
ax1.set(ylabel='Halpha/ FUV',xlabel='eq width')
#plt.savefig(basedir/'reports'/'age_vs_Ha_over_FUV.png',dpi=600)
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))

ax1.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],color='black')
#sc = ax1.scatter(FUV_int,Halpha,c=time_HA/1e6,vmin=0,vmax=3)
#fig.colorbar(sc,ax=ax1,label='age / Myr')
ax1.set(ylabel='Halpha/ FUV',xlabel='time / Myr',xlim=[0,10])
plt.savefig(basedir/'reports'/'age_vs_Ha_over_FUV.png',dpi=600)
plt.show()

### Filter response curve

to get the FUV flux by integrating the spectrum. The curves are from the [astrosat website](https://uvit.iiap.res.in/Instrument/Filters)



In [ ]:
from speclite.filters import FilterResponse, load_filters, plot_filters

response_curve = ascii.read(basedir/'data'/'external'/'astrosat_response_curve.txt',
                                     names=['wavelength','EA','Filter'])

F148W_mask = response_curve['Filter']=='F148W'
F148W_lam = response_curve['wavelength'][F148W_mask]*u.angstrom
F148W_res = response_curve['EA'][F148W_mask] / max(response_curve['EA'][F148W_mask])
F148W = FilterResponse(F148W_lam,F148W_res,meta=dict(group_name='Astrosat',band_name='F148W'))

F154W_mask = response_curve['Filter']=='F154W'
F154W_lam  = response_curve['wavelength'][F154W_mask]*u.angstrom
F154W_res  = response_curve['EA'][F154W_mask] / max(response_curve['EA'][F154W_mask])
F154W = FilterResponse(F154W_lam,F154W_res,meta=dict(group_name='Astrosat',band_name='F154W'))

astrosat_filter = load_filters('Astrosat-F148W', 'Astrosat-F154W')
plot_filters(astrosat_filter)

### Compare Halpha and FUV

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
Ha  = cluster.ewidth['Luminosity_H_A']
time_HA = cluster.ewidth['Time']

FUV  = cluster.FUV['FUV']
time_FUV = cluster.FUV['Time']

ax1.plot(time_HA/1e6,Ha,color='tab:red')
ax1.set_ylabel('Halpha/ (erg/s)',color='tab:red')
ax1.set(xlabel='Time/Myr')

ax2 = ax1.twinx() 
ax2.plot(time_FUV/1e6,FUV,color='tab:green')
ax2.set_ylabel('FUV / (erg/s)',color='tab:green')
ax2.set(xlabel='Time/Myr',xlim=[0,10])
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

FUV_int = np.interp(time_HA,time_FUV,FUV)

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))
ax1.plot(time_HA/1e6,Ha/FUV_int,color='tab:blue')
ax1.set_ylabel('Halpha / FUV',color='tab:blue')
ax1.set(xlim=[0,3],ylim=[1e-5,0.0017],xlabel='Time/Myr')

axt = ax1.twinx()
quanta = cluster.quanta
axt.plot(quanta['Time']/1e6,quanta['HI_rate'],color='tab:orange')
axt.set_ylabel('ionizing photons / 1/s',color='tab:orange')
axt.set(xlim=[0,10])

HI_rate_int = np.interp(time_HA,quanta['Time'],quanta['HI_rate'])

ax2.plot(HI_rate_int,Ha/FUV_int,color='black')
sc = ax2.scatter(HI_rate_int,Ha/FUV_int,c=time_HA/1e6,vmin=0,vmax=10)
divider = make_axes_locatable(ax2)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(sc,cax=cax,label='age / Myr',pad=-1)

ax2.set(xlabel='ionizing photons / 1/s',ylabel='Halpha / FUV')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))

ax1.plot(time_HA/1e6,Ha/FUV_int,color='black')
#sc = ax1.scatter(FUV_int,Halpha,c=time_HA/1e6,vmin=0,vmax=3)
#fig.colorbar(sc,ax=ax1,label='age / Myr')
ax1.set(ylabel='Halpha/ FUV',xlabel='time / Myr',xlim=[0,10])
plt.savefig(basedir/'reports'/'age_vs_Ha_over_FUV.png',dpi=600)
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(two_column,two_column/1.618))

for mass in [5e2,1e3,2e3,5e3,1e4,2e4,5e4]:
    
    scaled_cluster = cluster.scale(mass)
    
    Halpha  = scaled_cluster.ewidth['Luminosity_H_A']
    time_HA = scaled_cluster.ewidth['Time']
    FUV = scaled_cluster.FUV['FUV']
    
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    ax1.text(np.log10(FUV.value)[0],np.log10(Halpha.value)[0],f'{mass:.0g}  ',
            horizontalalignment='right',verticalalignment='bottom')
    sc = ax1.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax1.set(ylabel=r'log10 H$\alpha$ / (erg/s)',xlabel='log10 FUV / (erg/s)')
fig.colorbar(sc,label='age / Myr')
plt.show()

In [ ]:
clusters = {}
for m in [23,53,63,24,54,64]:
    clusters[m] = Cluster(stellar_model=m)

In [ ]:
fig,axes =plt.subplots(nrows=2,ncols=3,figsize=(two_column,two_column/1.618))
axes_iter = iter(axes.flatten())

for m in [23,53,63,24,54,64]:
    
    ax = next(axes_iter)
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    ax.set_title(label,fontsize=8)
    sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

    ax.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')

plt.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.1, 0.05, 0.7])
fig.colorbar(sc, cax=cbar_ax,label='age / Myr')

plt.show()

In [ ]:
fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))

for m in [23,53,63]:
    
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),label=label)
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    #sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax1.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')
ax1.legend()

for m in [24,54,64]:
    
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']

    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax2.plot(np.log10(FUV.value),np.log10(Halpha.value),label=label)
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    #sc = ax.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

ax2.set(ylabel=r'H$\alpha$ / (erg/s)',xlabel='FUV / (erg/s)')
ax2.legend()
    
plt.tight_layout()


plt.show()

In [ ]:
fig,axes =plt.subplots(nrows=2,ncols=3,figsize=(two_column,two_column/1.618))
axes_iter = iter(axes.flatten())

for m in [23,53,63,24,54,64]:
    
    ax = next(axes_iter)
    cl = clusters[m]
    
    Halpha  = cl.ewidth['Luminosity_H_A']
    time_HA = cl.ewidth['Time']
    FUV = cl.FUV['FUV']
    ionizing = cl.quanta['HI_rate']
  
    label = f'{cl.stellar_model}, Z={cl.metallicity}'
    ax.plot(Halpha/FUV,ionizing,color='black')
    #t = ax.text(0.05,0.9,label, transform=ax.transAxes,color='black',fontsize=8)
    ax.set_title(label,fontsize=8)
    sc = ax.scatter(Halpha/FUV,ionizing,c=time_HA/1e6,vmin=0,vmax=10)

    ax.set(ylabel=r'H$\alpha$ / FUV',xlabel='ionization')

plt.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.9, 0.1, 0.05, 0.7])
fig.colorbar(sc, cax=cbar_ax,label='age / Myr')

plt.show()

### Compare observations to simulations

In [ ]:
catalogue['distance'] = np.nan
for name in np.unique(catalogue['gal_name']):
    catalogue['distance'][catalogue['gal_name']==name] = (sample_table.loc[name]['Distance']*u.Mpc).to(u.cm)

In [ ]:
fig,ax1 =plt.subplots(figsize=(two_column,two_column/1.618))

for mass in [2e3,5e3,1e4,2e4,5e4,1e5,2e5,5e5,1e6,2e6,5e6]:
    
    scaled_cluster = cluster.scale(mass)
    
    Halpha  = scaled_cluster.ewidth['Luminosity_H_A']
    time_HA = scaled_cluster.ewidth['Time']
    FUV = scaled_cluster.FUV['FUV']
    
    ax1.plot(np.log10(FUV.value),np.log10(Halpha.value),color='black')
    ax1.text(np.log10(FUV.value)[0],np.log10(Halpha.value)[0],f'{mass:.0g}  ',
            horizontalalignment='right',verticalalignment='bottom')
    sc = ax1.scatter(np.log10(FUV.value),np.log10(Halpha.value),c=time_HA/1e6,vmin=0,vmax=30)

Halpha_FLUX = ((catalogue['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*catalogue['distance']**2).value
FUV_FLUX = 5e5*((catalogue['FUV_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*catalogue['distance']**2).value
ax1.scatter(np.log10(FUV_FLUX),np.log10(Halpha_FLUX))

ax1.set(ylabel=r'log10 H$\alpha$ / (erg/s)',xlabel='log10 FUV / (erg/s)')
fig.colorbar(sc,label='age / Myr')
plt.show()

create the grid to compare the observations to

In [ ]:
n_time = len(cluster.ewidth['Time'])
n_mass = 1000

mass_min = 5e3
mass_max = 1e6

mass_grid = np.linspace(mass_min,mass_max,n_mass)

HA_grid = np.zeros((n_time,n_mass))
FUV_grid = np.zeros((n_time,n_mass))

for i,mass in enumerate(mass_grid):
    
    scaled_cluster = cluster.scale(mass)
    
    HA_grid[:,i]  = scaled_cluster.ewidth['Luminosity_H_A']
    FUV_grid[:,i] = scaled_cluster.FUV['FUV']

time = scaled_cluster.FUV['Time']

In [ ]:
#distance = (sample_table.loc[name]['Distance']*u.Mpc).to(u.cm)

mass, age, chi2 = [],[],[]
for row in catalogue:
    
    Halpha_FLUX = ((row['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    Halpha_ERR  = ((row['HA6562_FLUX_CORR_ERR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    
    FUV_FLUX = 1e6*((row['FUV_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    FUV_ERR  = ((row['FUV_FLUX_CORR_ERR']*1e-20*u.erg/u.s/u.cm**2) * 4*np.pi*row['distance']**2).value
    
    chi2_grid = (Halpha_FLUX-HA_grid)**2/Halpha_ERR**2 + (FUV_FLUX-FUV_grid)**2/FUV_ERR**2
    
    row,col = np.unravel_index(chi2_grid.argmin(), chi2_grid.shape)
    mass.append(mass_grid[col])
    age.append(time[row].value)
    chi2.append(np.min(chi2_grid))
    

In [ ]:
fig,(ax1,ax2) =plt.subplots(nrows=1,ncols=2,figsize=(two_column,two_column/2))

ax1.scatter(np.array(age)/1e6,catalogue['AGE_MINCHISQ'])
ax1.set(ylim=[0,30],xlim=[0,30],xlabel='age from Nebulae / Myr',ylabel='age from Cluster / Myr')

ax2.scatter(np.array(mass),catalogue['MASS_MINCHISQ'])
ax2.set(xlabel='mass from Nebulae / Msun',ylabel='mass from Cluster / Msun')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(nrows=1,ncols=1,figsize=(single_column,single_column))

ax1.scatter(np.array(age)/1e6,np.array(mass))
ax1.set(xlim=[0,30],ylim=[0,5e5],xlabel='age from Nebulae / Myr',ylabel='mass / Msun')

plt.tight_layout()
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
eqwHA  = cluster.ewidth['Equ_width_H_A']
time_HA = cluster.ewidth['Time']

ax1.plot(time_HA/1e6,eqwHA,color='tab:red')
ax1.set(xlabel='Time/Myr',ylabel='eq / AA',xlim=[0,10])

#ax2 = ax1.twinx() 
#ax2.plot(time_FUV/1e6,FUV,color='tab:green')
#ax2.set_ylabel('FUV / (erg/s)',color='tab:green')
#ax2.set(xlabel='Time/Myr',xlim=[0,10])
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
colors = ['tab:blue','tab:cyan','tab:red','tab:orange','tab:green','tab:olive']

for m,c in zip([23,24,53,54,63,64],colors):
    
    cl = clusters[m]
    
    eqwHA  = cl.ewidth['Equ_width_H_A']
    time_HA = cl.ewidth['Time']
    ax1.plot(np.log10(time_HA/u.yr),np.log10(eqwHA/u.angstrom),label=f'{cl.stellar_model }',color=c)
    
    #sl = cl.scale(1e5)
    #eqwHA  = sl.ewidth['Equ_width_H_A']
    #time_HA = sl.ewidth['Time']
    #ax1.plot(np.log10(time_HA/u.yr),np.log10(eqwHA/u.angstrom),ls='--',color=c)
    
    
ax1.set(xlabel='log (Time / Myr)',ylabel=r'log (W(H$\alpha$) / $\AA$)',xlim=[6,7.5])
ax1.legend()
#plt.savefig(basedir/'reports'/'equivalent_width_vs_age.pdf',dpi=600)
plt.show()

In [ ]:
fig,ax1 =plt.subplots(figsize=(single_column,single_column))
colors = ['tab:blue','tab:cyan','tab:red','tab:orange','tab:green','tab:olive']

for m,c in zip([23,24,53,54,63,64],colors):
    
    cl = clusters[m]
    
    HAFUV  = cl.ewidth['Luminosity_H_A'] / cl.FUV['FUV']
    time_HA = cl.ewidth['Time']
    ax1.plot(time_HA/1e6,HAFUV,label=f'{cl.stellar_model }',color=c)
    
    sl = cl.scale(1e5)
    HAFUV  = sl.ewidth['Luminosity_H_A'] / sl.FUV['FUV']
    time_HA = sl.ewidth['Time']
    ax1.plot(time_HA/1e6,HAFUV,ls='--',color=c)    
    
ax1.set(xlabel='Time / Myr',ylabel=r'HA/FUV',xlim=[0,10])
ax1.legend()
#plt.savefig(basedir/'reports'/'equivalent_width_vs_age.pdf',dpi=600)
plt.show()

### Create custom cluster

In [ ]:
def create_cluster(folder,**kwargs):

    parameters = {
    "name" : 'standard',
    "isf" : -1,
    "mass" : 1.,
    "sfr" : 1,
    "ninterv" : 2,
    "xponent" : '1.3,2.3',
    "xmaslim" : '0.1,0.5,120',
    "sncut" : 8.,
    "bhcut" : 120.,
    "model" : 64,
    "wind_model" : 0,
    "tinitial" : 0.01,
    "time_scale" : 0,
    "time_step" : 0.1,
    "n_steps" : 1000,
    "tmax" : 50,
    "jmg" : 3,
    "atmos" : 5,
    "metallicity" : 3,
    "uvline" : 1}

    # assign the new parameters
    for k,v in kwargs.items():
        if k in parameters:
            parameters[k] = v
    
    # open tempalte
    with open(basedir/'data'/'input.template') as f:
        template = f.read()
    
    template = template.format(**parameters)

    #write to templae
    with open(folder/'input.out','w') as f:
        f.write(template)
        
        


## Playground

### Principal component analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
columns = ['age','mass','HA/FUV','HA6562_FLUX','region_area']


data = np.zeros((len(catalogue),len(columns)))

for i,col in enumerate(columns):
    data[:,i] = catalogue[col].data
data = StandardScaler().fit_transform(data)

In [ ]:
pca_nebulae = PCA(n_components=2)
principalComponents = pca_nebulae.fit_transform(data)